In [ ]:
from IPython.display import display

import pandas as pd
import warnings
from model_drift import settings, helpers
from model_drift.data.utils import nested2series
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
from model_drift.drift.tabular import TabularDriftCalculator
from model_drift.drift.numeric import KSDriftCalculator, BasicDriftCalculator
from model_drift.drift.categorical import ChiSqDriftCalculator
from model_drift.drift.collection import DriftCollectionCalculator
from model_drift.drift.performance import AUROCCalculator

from model_drift.drift.sampler import Sampler
from model_drift.data.padchest import PadChest
import plotly.graph_objects as go

warnings.filterwarnings("ignore")


In [ ]:
jsonl_file = str(settings.TOP_DIR.joinpath("results", 'vae', 'padchest-trained', 'preds.jsonl'))
vae_df = helpers.jsonl_files2dataframe(jsonl_file)
vae_df = pd.concat(
    [
        vae_df,
        pd.DataFrame(vae_df['mu'].values.tolist(), columns=[f"mu.{c}" for c in range(128)])
    ],
    axis=1
)
vae_df.head()

In [ ]:
from model_drift.data.padchest import LABEL_MAP
label_cols = list(LABEL_MAP)
jsonl_file = str(settings.TOP_DIR.joinpath("results", 'classifier', 'finetuned', "preds.jsonl"))
scores_df = helpers.jsonl_files2dataframe(jsonl_file)
scores_df = pd.concat(
    [
        scores_df,
        pd.DataFrame(scores_df['activation'].values.tolist(), columns=[f"activation.{c}" for c in label_cols])
    ],
    axis=1)
scores_df.head()


In [ ]:
# Load padchest CSV
pc = PadChest(settings.PADCHEST_FILENAME)
pc.prepare()


pc.merge(vae_df, left_on="ImageID", right_on="index", how='inner')
pc.merge(scores_df, left_on="ImageID", right_on="index", how='inner')


train, val, test = pc.split(settings.PADCHEST_SPLIT_DATES, studydate_index=True)


In [ ]:
pd.concat(
    {
        "all": pc.df["StudyDate"].describe(datetime_is_numeric=True),
        "train": train.df["StudyDate"].describe(datetime_is_numeric=True),
        "val": val.df["StudyDate"].describe(datetime_is_numeric=True),
        "test": test.df["StudyDate"].describe(datetime_is_numeric=True),
    },
    axis=1,
)


In [ ]:
FLOAT = KSDriftCalculator
CAT = ChiSqDriftCalculator

cols = {
    'age': FLOAT,
    'Projection': CAT,
    "PatientSex_DICOM": CAT,
    "ViewPosition_DICOM": CAT,
    "Modality_DICOM": CAT,
    "Manufacturer_DICOM": CAT,
    "PhotometricInterpretation_DICOM": CAT,
    "PixelRepresentation_DICOM": CAT,
    "PixelAspectRatio_DICOM": CAT,
    "SpatialResolution_DICOM": CAT,
    "BitsStored_DICOM": CAT,
    "WindowCenter_DICOM": FLOAT,
    "WindowWidth_DICOM": FLOAT,
    "Rows_DICOM": FLOAT,
    "Columns_DICOM": FLOAT,
    "XRayTubeCurrent_DICOM": CAT,
    "Exposure_DICOM": CAT,
    "ExposureInuAs_DICOM": FLOAT,
    "RelativeXRayExposure_DICOM": FLOAT,
    'Frontal': BasicDriftCalculator,
}

cols.update({c:FLOAT for c in list(pc.df) if c.startswith("mu.") and 'all' not in c})
cols.update({c:FLOAT for c in list(pc.df) if c.startswith("activation.") and 'all' not in c})
cols[("score", "label")] = AUROCCalculator


In [ ]:
window = "30D"
stride = "D"
ref_frontal_only = True
min_periods = 150

nonfrontal_add_date = "2015-01-01"
frontal_remove_date = "2016-06-01"

replacement = True
sample_size = 2000
n_samples = 20


sampler = Sampler(sample_size, replacement=replacement)


In [ ]:
refdf = val.df.copy()
if ref_frontal_only:
    refdf = refdf.query("Frontal")


print(len(refdf), len(val.df))

dwc = TabularDriftCalculator(refdf)

for c, kls in cols.items():
    dwc.add_drift_stat(c, kls)
dwc.prepare()

target_df = pc.df.set_index('StudyDate')


nonfrontals_target_df = target_df.query("~Frontal")
frontals_target_df = target_df.query("Frontal")


target_df = val.df.copy()#.query("Frontal")


# print(target_df['Frontal'].mean())
# target_df = pd.concat([frontals_target_df.loc[:frontal_remove_date],
#                       nonfrontals_target_df.loc[nonfrontal_add_date:]]).sort_index()
# print(target_df['Frontal'].mean())



print(len(target_df), len(pc.df.set_index('StudyDate')))


In [ ]:
fname = settings.TOP_DIR.joinpath(
    "results", "drift_csvs", f"valwithnonfrontals-s{stride}-w{window}-min{min_periods}_frontalonly-ref{ref_frontal_only}_Samp-ss{sample_size}-n{n_samples}-repl{replacement}.csv")
print(fname)


In [ ]:
from model_drift.data.utils import rolling_window_dt_apply

frontal_over_time = rolling_window_dt_apply(target_df, lambda x: {'frontal': x['Frontal'].mean()}, n_jobs=5, backend='threading')

frontal_over_time.plot(y='frontal', figsize=(20, 8))


In [ ]:
output = dwc.rolling_window_predict(target_df,
                                    sampler=sampler, n_samples=n_samples,
                                    stride=stride, window=window, min_periods=min_periods,
                                    n_jobs=5, backend="threading"
                                    )


In [ ]:
print(fname)
output.to_csv(fname)


In [ ]:
output.head()

In [ ]:


output['score'].ewm(span=90).mean().plot(figsize=(20, 8))

frontal_over_time.ewm(span=90).mean().plot(y='frontal', figsize=(20, 8))

frontal_over_time.ewm(span=90).mean().plot(y='window_count', figsize=(20, 8))


In [ ]:
output[["Frontal", 'score']].drop([("Frontal", 'stats', "std")], axis=1).plot(figsize=(20, 8))
